# AI Language #

See coding instructions given on paper from instructor.

1.   Write the entropy function for binary classification from scratch using only Python's math module. DO NOT USE NUMPY
2.   Do NOT call built-in entropy function from any Python library.
3. Use the math library only. Do not use any other library.
4. You have to write the entropy function from scratch on your own.
5. Your entropy function should take a probability value for one of the two calsses as input, and output its entropy value.
6. Call your entropy function using 0.2 as input.


In [ ]:
#Math Library
import math

**-Replace xxx with the appropriate value**

**Q4-0-0:** Print the output of the function call as "The entorpy value of probability 0.2 is xxx"

7. Call your entropy function using 0.8 as input

**Q4-0-1** Print the outut of the function call as "The entropy value of probability 0.8 is xxx"

8. Call your entropy function using 0.5 as input

**Q4-0-2** Print the output of the function call as "The entropy value of probability 0.5 is xxx"

In [ ]:
#defining entropy for a binary classification
def entropy(p):
  # Handle cases where p is 0 or 1
    if p == 0 or p == 1:
        return 0
    # Calculate entropy using the full formula
    return -p * math.log2(p) - (1 - p) * math.log2(1 - p)

#calling the entropy function with p=0.2
result = entropy(0.2)

#Q4-0-0: print the output of the function call as' The entropy value of probability 0.2 is xxx'
print("The entropy value of probability 0.2 is", result)

#calling the entropy function with p=0.8
result1 = entropy(0.8)

#Q4-0-1: print the output of the function call as' The entropy value of probability 0.8 is xxx'
print("The entropy value of probability 0.8 is", result1)

#calling the entropy function with p=0.5
result2 = entropy(0.5)

#Q4-0-2: print the output of the function call as' The entropy value of probability 0.5 is xxx'
print("The entropy value of probability 0.5 is", result2)

The entropy value of probability 0.2 is 0.7219280948873623
The entropy value of probability 0.8 is 0.7219280948873623
The entropy value of probability 0.5 is 1.0


# M4 Assignment 1: RNN for Time Series Prediction

This assignment is based on the Time Series Prediction lab 10.9.6 from ISLP Chapter 10.



*   Please use the texbook lab as a reference
*   You should write your model using Tensorflow
*   The goal is to predict **log_volume** using lagged data



**Step 0: Load Libraries**

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import random
import numpy as np
import tensorflow as tf
import pandas as pd


random.seed(1693)
np.random.seed(1693)
tf.random.set_seed(1693)

**Step 1: Load & Prep Dataframe**



1.   Load the NYSE datset from the NYSE.csv file available
2.   The data column gives you the timestamps of the time series
3.   Train column indicates True for records to be used in the train set, and False for those to be used in the test set
4.   For this step, let's keep only these 3 columns: 'DJ_return', 'log_volune', 'log_volatility'
5.   Standardize all 3 columns using ScikitLearn's StandardScaler

In the starter code given below:
*   cols is a list of the names of these 3 columns
*   X is a dataframe that contains only these 3 columns from NYSE.csv



In [ ]:
#Load the NYSE dataset
df = pd.read_csv('NYSE.csv')

#Define the columns to keep
cols = ['DJ_return', 'log_volume', 'log_volatility']

#Create a new copy with only the specified columns
X = df[cols].copy()

#Standardize the columns using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[cols] = scaler.fit_transform(X[cols])

#Q4-1-0 Print '0. the shape of dataframe X:xxx
print(f'the shape of dataframe X:', X.shape)

#4-1-1 Print '1. the first record of datarame X:xxx
print(f'the first record of dataframe X:', X.iloc[0])


the shape of dataframe X: (6051, 3)
the first record of dataframe X: DJ_return        -0.549823
log_volume        0.175075
log_volatility   -4.357078
Name: 0, dtype: float64


Use code from the textbook lab to set up lagged versions of these 3 data columns.

Add column 'train' from the orginial dataset to the current dataframe X as the last column (to the right)

In [ ]:
#Add Lagged versions of the 3 columns
lags = 5
for lag in range(1, lags + 1):
  for col in cols:
    X[f'{col}_lag_{lag}'] = X[col].shift(lag)

#Add the 'train' column from the original dataset to X
X['train'] = df['train']

#Q4-1-2 Print '2. the shape of dataframe X with lags:xxx
print(f'the shape of dataframe X with lags:', X.shape)

#Q4-1-3 Print '3. the first record of dataframe X with lags:xxx
print(f'the first record of dataframe X with lags:', X.iloc[0])

#Drop rows with missing values
X = X.dropna()

#Q4-1-4 Print '4. the shape of dataframe X with lags:xxx
print(f'the shape of dataframe X with lags:', X.shape)

#Q4-1-5 Print '5. the first record of dataframe X with lags:xxx
print(f'the first record of dataframe X with lags:', X.iloc[0])


the shape of dataframe X with lags: (6051, 19)
the first record of dataframe X with lags: DJ_return              -0.549823
log_volume              0.175075
log_volatility         -4.357078
DJ_return_lag_1              NaN
log_volume_lag_1             NaN
log_volatility_lag_1         NaN
DJ_return_lag_2              NaN
log_volume_lag_2             NaN
log_volatility_lag_2         NaN
DJ_return_lag_3              NaN
log_volume_lag_3             NaN
log_volatility_lag_3         NaN
DJ_return_lag_4              NaN
log_volume_lag_4             NaN
log_volatility_lag_4         NaN
DJ_return_lag_5              NaN
log_volume_lag_5             NaN
log_volatility_lag_5         NaN
train                       True
Name: 0, dtype: object
the shape of dataframe X with lags: (6046, 19)
the first record of dataframe X with lags: DJ_return              -1.304126
log_volume              0.605918
log_volatility         -1.366028
DJ_return_lag_1          0.04634
log_volume_lag_1        0.224779
log_v

* Create the Y response target using the 'log_volume' column from dataframe X
* Extract the 'train' column from dataframe X as a separate variable called train. Drop the 'train' column from dataframe X
* Later on we will use the train variable to split the dataset into train vs test
* Drop the current day's DJ_return (the 'DJ_return' column) and log_volatility from dataframe X
* In other words, remove these two X features, and also the Y response that came from dataframe X

In [ ]:
#print(X.columns)

# Create the Y response target using the 'log_volume' column from dataframe X
Y = X['log_volume'].copy()

# Extract the 'train' column from dataframe X as a separate variable called train
train = X['train'].copy()  # train is a separate variable for splitting the dataset

# Drop the 'train' column from dataframe X
X = X.drop(columns=['train'])

# Drop the current day's 'DJ_return' and 'log_volatility' from dataframe X
X = X.drop(columns=['DJ_return', 'log_volatility'])

#Q4-1-6 Print '6. the first 3 records of the Y target:xxx'
print(f'the first 3 records of the Y target:', Y.iloc[:3])

#Q4-1-7 Print '7. the first 3 records of the train variable:xxx'
print(f'the first 3 records of the train variable:', train.iloc[:3])

#Q4-1-8 Print '8. the first 3 records of dataframe X:xxx'
print(f'the first 3 records of dataframe X:', X.iloc[:3])

#Q4-1-8 Print '8. the first 3 records of the X features:xxx'
print(f'the first 3 records of the X features:', X.iloc[:3])

the first 3 records of the Y target: 5    0.605918
6   -0.013661
7    0.042552
Name: log_volume, dtype: float64
the first 3 records of the train variable: 5    True
6    True
7    True
Name: train, dtype: bool
the first 3 records of dataframe X:    log_volume  DJ_return_lag_1  log_volume_lag_1  log_volatility_lag_1  \
5    0.605918         0.046340          0.224779             -2.500970   
6   -0.013661        -1.304126          0.605918             -1.366028   
7    0.042552        -0.006294         -0.013661             -1.505667   

   DJ_return_lag_2  log_volume_lag_2  log_volatility_lag_2  DJ_return_lag_3  \
5        -0.431397          0.935176             -2.366521         0.434813   
6         0.046340          0.224779             -2.500970        -0.431397   
7        -1.304126          0.605918             -1.366028         0.046340   

   log_volume_lag_3  log_volatility_lag_3  DJ_return_lag_4  log_volume_lag_4  \
5          2.283789             -2.418037         0.905200  

To fit the RNN, we must reshape the X dataframe, as the RNN layer will expect 5 lagged versions of each feature as indicated by the (5,3) shape of the RNN layer below.

1. We first ensure the columns of our X dataframe are such that a reshaped matrix will have the variables correctly lagged. We use the reindex() method to do this.

The RNN layer also expects the first row of each observation to be earliest in time. So we must revers the current order.

Follow the textbook lab code to reorder/reindex the columns properly.

In [ ]:
# Ensure the columns of X are in the correct order for reshaping
ordered_cols = []
for lag in range(5, 0, -1):  # Loop from lag 5 to lag 1 (reverse order)
    for col in cols:  # Loop through each feature
        ordered_cols.append(f'{col}_lag_{lag}')  # Add the lagged column to the ordered list

# Reindex the dataframe X to the ordered columns
X = X.reindex(columns=ordered_cols)

#Q4-1-9 Print '9. the first 3 records of X after reindexting: xxx'
print(f'the first 3 records of X after reindexting:', X.iloc[:3])

the first 3 records of X after reindexting:    DJ_return_lag_5  log_volume_lag_5  log_volatility_lag_5  DJ_return_lag_4  \
5        -0.549823          0.175075             -4.357078         0.905200   
6         0.905200          1.517291             -2.529058         0.434813   
7         0.434813          2.283789             -2.418037        -0.431397   

   log_volume_lag_4  log_volatility_lag_4  DJ_return_lag_3  log_volume_lag_3  \
5          1.517291             -2.529058         0.434813          2.283789   
6          2.283789             -2.418037        -0.431397          0.935176   
7          0.935176             -2.366521         0.046340          0.224779   

   log_volatility_lag_3  DJ_return_lag_2  log_volume_lag_2  \
5             -2.418037        -0.431397          0.935176   
6             -2.366521         0.046340          0.224779   
7             -2.500970        -1.304126          0.605918   

   log_volatility_lag_2  DJ_return_lag_1  log_volume_lag_1  \
5      

Reshape dataframe X as a 3-D Numpy array such that each record/ row has the shape of (5,3). Each row represents a lagged version of the 3 variables in the shape of (5,3)

In [ ]:
# Ensure the columns of X are in the correct order for reshaping
ordered_cols = []
for lag in range(1, 6):  # Loop from lag 1 to lag 5
    for col in cols:  # Loop through each feature
        ordered_cols.append(f'{col}_lag_{lag}')  # Add the lagged column to the ordered list

# Reindex the dataframe X to the ordered columns
X = X.reindex(columns=ordered_cols)

# Reshape X to the shape (samples, timesteps, features)
X_rnn = X.to_numpy().reshape((-1, 5, 3))  # Shape: (samples, 5, 3)

#Q4-1-10: Print '10. the shape of X after reshaping: xxx
print(f'the shape of X after reshaping:', X_rnn.shape)

#Q4-1-11: Print '11. the first 2 records of X after reshaping: xxx
print(f'the first 2 records of X after reshaping:', X_rnn[:2])

the shape of X after reshaping: (6046, 5, 3)
the first 2 records of X after reshaping: [[[ 0.04634026  0.22477858 -2.5009701 ]
  [-0.43139673  0.93517558 -2.36652094]
  [ 0.43481275  2.28378937 -2.41803694]
  [ 0.90519995  1.51729071 -2.52905765]
  [-0.54982334  0.17507497 -4.35707786]]

 [[-1.30412619  0.60591805 -1.366028  ]
  [ 0.04634026  0.22477858 -2.5009701 ]
  [-0.43139673  0.93517558 -2.36652094]
  [ 0.43481275  2.28378937 -2.41803694]
  [ 0.90519995  1.51729071 -2.52905765]]]


 **Step 2: Prep Train vs Test Sets**

 * Set up my X_train, X_test, Y_train, and Y_test using the X dataframe, Y response target, and the train variable you have created above

 *Include recors where train = True in the train set, and train = False in the test set

In [ ]:
# Split the dataset into train and test sets using the train variable
X_train = X_rnn[train]  # X_train includes records where train = True
X_test = X_rnn[~train]  # X_test includes records where train = False

Y_train = Y[train]  # Y_train includes records where train = True
Y_test = Y[~train]  # Y_test includes records where train = False

**Step 3: Train Model**

Confirgure a Keras Sequential model with
1. proper input shape
2. SimpleRNN layer with 12 hidden units
3. relu activation function
4. 10% dropout
5. proper output layer

Do not name the model or any of the layers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense

# Configure the Keras Sequential model
model = Sequential([
    SimpleRNN(12, activation='relu', input_shape=(5, 3)),  # SimpleRNN layer with 12 hidden units and relu activation
    Dropout(0.1), # 10% dropout
    Dense(1)  # Output layer with 1 unit (for regression)
])

# Print the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 12)                  │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 12)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              13 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 205 (820.00 B)

 Trainable params: 205 (820.00 B)

 Non-trainable params: 0 (0.00 B)

Compile the model with

1. adam optimizer
2. MSE as the loss
3. MSE as the metric

Fit the model with

1. 200 epochs
2. batch size of 32

No need to print epoch-by epoch progress

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',  # Adam optimizer
    loss='mse',        # Mean Squared Error (MSE) as the loss function
    metrics=['mse']    # Mean Squared Error (MSE) as the metric
)

# Fit the model
history = model.fit(
    X_train, Y_train,          # Training data
    epochs=200,                # Number of epochs
    batch_size=32,             # Batch size
    validation_data=(X_test, Y_test),  # Validation data
    verbose=0                  # Suppress epoch-by-epoch progress
)

**Step 4: Evaluate Model**

Evaluate the model using model.evaluate() with the test set

In [ ]:
# Evaluate the model on the test set
loss, mse = model.evaluate(X_test, Y_test, verbose=0)

#Q4-1-13 Print '13. Test MSE: xxx
print(f'Test MSE:', mse)

Test MSE: 0.6517770290374756
